In [6]:
import numpy as np
import pandas as pd
from sklearn.ensemble import RandomForestClassifier
from sklearn.inspection import permutation_importance
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix, roc_auc_score
from sklearn.model_selection import train_test_split

In [ ]:
# Import data
data_new = pd.read_pickle(r'data\04_fct\fct_demographic_offers_and_transactions.pkl')

In [ ]:
def process_data(data, gender_column):
    # Filter data for the specified gender
    gender_data = data[data[gender_column] == 1]
    # Extract features and target variable
    features = gender_data[['mobile', 'social', 'web']]
    target = gender_data['offer_viewed']
    # Split the data into training and testing sets
    X_train, X_test, y_train, y_test = train_test_split(features, target, test_size=0.3, random_state=42)
    return X_train, X_test, y_train, y_test

def train_evaluate_model(X_train, X_test, y_train, y_test):
    # Initialize and train the random forest classifier
    rf_clf = RandomForestClassifier(random_state=42)
    rf_clf.fit(X_train, y_train)
    # Make predictions
    y_pred = rf_clf.predict(X_test)
    y_prob = rf_clf.predict_proba(X_test)[:, 1]
    # Calculate metrics
    accuracy = accuracy_score(y_test, y_pred)
    conf_matrix = confusion_matrix(y_test, y_pred)
    class_report = classification_report(y_test, y_pred)
    roc_auc = roc_auc_score(y_test, y_prob)
    # Calculate permutation importance
    perm_importance = permutation_importance(rf_clf, X_test, y_test, n_repeats=10, random_state=42)
    perm_importance_df = pd.DataFrame({'Feature': ['mobile', 'social', 'web'],
                                       'Importance': perm_importance.importances_mean,
                                       'Importance STD': perm_importance.importances_std})
    return {
        "Accuracy": accuracy,
        "Confusion Matrix": conf_matrix,
        "Classification Report": class_report,
        "ROC-AUC Score": roc_auc,
        "Permutation Importance": perm_importance_df
    }

In [2]:
# Process data for female and male customers
X_train_female, X_test_female, y_train_female, y_test_female = process_data(data_new, 'gender_F')
X_train_male, X_test_male, y_train_male, y_test_male = process_data(data_new, 'gender_M')

# Train and evaluate models for female and male customers
results_female = train_evaluate_model(X_train_female, X_test_female, y_train_female, y_test_female)
results_male = train_evaluate_model(X_train_male, X_test_male, y_train_male, y_test_male)

{'Accuracy': 0.7986263334794681,
 'Confusion Matrix': array([[ 427, 1115],
        [ 263, 5038]], dtype=int64),
 'Classification Report': '              precision    recall  f1-score   support\n\n           0       0.62      0.28      0.38      1542\n           1       0.82      0.95      0.88      5301\n\n    accuracy                           0.80      6843\n   macro avg       0.72      0.61      0.63      6843\nweighted avg       0.77      0.80      0.77      6843\n',
 'ROC-AUC Score': 0.8112081977533545,
 'Permutation Importance':   Feature  Importance  Importance STD
 0  mobile    0.029373        0.002633
 1  social    0.014000        0.002043
 2     web    0.003829        0.001550}

Conclusion
* Mobile Channel: More important for both genders but slightly more important for males.
* Social Channel: More important for males than females.
* Web Channel: Shows some importance for females but no importance for males.

- **ROC-AUC Score**: 0.8404

### Analysis

1. **Accuracy**:
 - The accuracy is similar for both genders, with females at 79.86% and males at 79.33%.

2. **Confusion Matrix**:
 - Both genders show a higher number of false positives (1115 for females and 1653 for males) compared to false negatives (263 for females and 309 for males).
 - The model performs well in identifying true positives for both genders.

3. **Classification Report**:
 - **Precision**: Slightly higher for males (0.81) compared to females (0.82) for the positive class.
 - **Recall**: Higher for both genders for the positive class, with males at 0.96 and females at 0.95.
 - **F1-Score**: Consistently higher for the positive class for both genders.

4. **ROC-AUC Score**:
 - The ROC-AUC score is higher for males (0.8404) compared to females (0.8112), indicating better performance in distinguishing between classes for male customers.

### Conclusion

- The model performs similarly for both genders, with slight differences in precision, recall, and ROC-AUC scores.
- The mobile and social channels are the most important for both genders, with the mobile channel being slightly more important for males.
- The web channel has limited importance overall, with some importance for females but none for males.
